In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

DATA_FILE_NAME = "data/train.csv"
df = pd.read_csv(DATA_FILE_NAME)
data_size=len(df)
df

AnimalID        Name             DateTime      OutcomeType  \
0      A671945     Hambone  2014-02-12 18:22:00  Return_to_owner   
1      A656520       Emily  2013-10-13 12:44:00       Euthanasia   
2      A686464      Pearce  2015-01-31 12:28:00         Adoption   
3      A683430         NaN  2014-07-11 19:09:00         Transfer   
4      A667013         NaN  2013-11-15 12:52:00         Transfer   
5      A677334        Elsa  2014-04-25 13:04:00         Transfer   
6      A699218       Jimmy  2015-03-28 13:11:00         Transfer   
7      A701489         NaN  2015-04-30 17:02:00         Transfer   
8      A671784        Lucy  2014-02-04 17:17:00         Adoption   
9      A677747         NaN  2014-05-03 07:48:00         Adoption   
10     A668402         NaN  2013-12-05 15:50:00         Transfer   
11     A666320         NaN  2013-11-04 14:48:00         Adoption   
12     A684601      Rocket  2016-02-03 11:27:00         Adoption   
13     A704702     Scooter  2015-06-08 16:30:00  Return_to_owner   
14     A688584     Preston  2015-11-25 15:00:00  Return_to_owner   
15     A678825      Oliver  2014-07-12 12:10:00         Adoption   
16     A678050         NaN  2014-05-03 16:15:00         Transfer   
17     A680028         NaN  2014-06-07 12:54:00         Transfer   
18     A679010     Chrissy  2014-05-17 11:32:00         Transfer   
19     A684787        Kuma  2014-07-30 17:34:00  Return_to_owner   
20     A670658      Anders  2014-01-19 15:03:00         Transfer   
21     A708812    Munchkin  2015-09-18 15:19:00         Transfer   
22     A705262     Jamaica  2015-08-15 14:22:00         Adoption   
23     A665781     Quattro  2013-10-28 16:32:00         Adoption   
24     A676333     Trigger  2014-04-09 17:44:00  Return_to_owner   
25     A712750       Olive  2015-10-03 15:44:00         Transfer   
26     A718832         Nia  2016-01-15 17:31:00         Adoption   
27     A698777       Chloe  2015-03-25 18:50:00         Adoption   
28     A715348     Cymbale  2015-11-21 13:01:00         Adoption   
29     A708666         Jax  2015-07-30 14:30:00  Return_to_owner   
...        ...         ...                  ...              ...   
26699  A676460       Mario  2014-04-21 14:01:00       Euthanasia   
26700  A704999        Lion  2015-06-15 19:28:00  Return_to_owner   
26701  A678842        Ella  2014-06-15 17:41:00         Transfer   
26702  A713637         NaN  2015-10-11 09:42:00             Died   
26703  A716670      Mowmow  2015-12-04 12:22:00         Transfer   
26704  A716223       Alice  2015-11-17 17:17:00         Transfer   
26705  A662498       Grace  2013-10-19 15:34:00         Transfer   
26706  A689942        Zoey  2014-10-19 13:29:00         Adoption   
26707  A682690         NaN  2014-07-01 17:06:00       Euthanasia   
26708  A666920      Herbie  2013-11-13 17:32:00         Transfer   
26709  A714386         NaN  2015-10-24 00:00:00         Adoption   
26710  A692622      Bonnie  2014-11-24 17:21:00         Transfer   
26711  A666284       Sonja  2013-10-30 18:32:00         Transfer   
26712  A535277      Merlin  2015-04-20 16:04:00  Return_to_owner   
26713  A671169         NaN  2014-01-20 17:37:00       Euthanasia   
26714  A679031        Pete  2014-05-31 16:11:00         Adoption   
26715  A708833        Luna  2015-08-05 17:03:00         Adoption   
26716  A697158         Max  2015-05-02 21:04:00         Adoption   
26717  A519949       Bella  2014-06-30 17:34:00  Return_to_owner   
26718  A701044         NaN  2015-04-28 14:26:00         Adoption   
26719  A707871         NaN  2015-07-20 09:00:00         Transfer   
26720  A704405     Brownie  2015-07-18 14:08:00         Adoption   
26721  A662019  Green Bean  2014-07-17 09:43:00         Adoption   
26722  A687056         NaN  2014-08-31 09:00:00         Transfer   
26723  A716997        Zeus  2016-01-29 18:52:00         Adoption   
26724  A702446         NaN  2015-05-14 11:56:00         Transfer   
26725  A718934         NaN  2016-01-20 18:59:00         Transf

In [2]:
df = df.drop(['AnimalID','Name','DateTime','OutcomeSubtype'], axis=1)
df['Breed_1'] = ""
df['Breed_2'] = ""
df['Color_1'] = ""
df['Color_2'] = "" 
df['isPurebred'] = ""
df['ReproductiveStatus'] = ""
def getAge(weeks):
    if weeks <= 0:
        return 'Unknown'
    elif weeks < 26:
        return 'Baby'
    elif weeks < 110:
        return 'Adolescent'
    elif weeks < 390:
        return 'Adult'
    else:
        return 'Senior'

def convertTargetType(value):
    if 'Adoption' in value:
        return 0
    elif 'Return_to_owner' in value:
        return 1
    elif 'Transfer' in value:
        return 2
    elif 'Euthanasia' in value:
        return 3
    else: # Died
        return 4
    
def findComplexDummies(series1,series2,prefix):
    series1 = prefix + '_' + series1
    series2 = prefix + '_' + series2
    possValues = sorted(list(set(series1.unique()).union(series2.unique())))
    #dummyFrame =  pd.DataFrame(0,index = np.arange(series1.size),columns=possValues)
    dummyFrame =  pd.DataFrame(np.zeros(shape=(len(series1),len(possValues))),columns=possValues)
    for index, value in series1.iteritems(): 
        dummyFrame[value][index] = 1
    for index, value in series2.iteritems(): 
        dummyFrame[value][index] = 1
    return dummyFrame
        
        
for index, row in df.iterrows():
    age = str(row['AgeuponOutcome'])
    if 'nan' in age:
        age = -1
    elif 'day' in age:
        age = 1
    elif 'week' in age:
        age = int(age.split(" ")[0])
    elif 'month' in age:
        age = 4 * int(age.split(" ")[0])
    elif 'year' in age:
        age = 52 * int(age.split(" ")[0])
    df['AgeuponOutcome'][index] = getAge(age)
    
    
    #Sex/Reproductive
    cur_sex = str(row['SexuponOutcome'])
    if "Unknown" not in cur_sex:
        sex_lst = cur_sex.split(" ")
        if ('Spayed' in sex_lst[0]) or ('Neutered' in sex_lst[0]):
            df['ReproductiveStatus'][index] = 'Neutered'
        else:
            df['ReproductiveStatus'][index] = 'Intact'
        try:
            df['SexuponOutcome'][index] = sex_lst[1]
        except:
            sex_lst
    else:
        df['ReproductiveStatus'][index] = 'Unknown'
        df['SexuponOutcome'][index] = 'Unknown'
    
    ##Breed
    cur_breed=row['Breed']
    if ('/' not in cur_breed) and ('Mix' not in cur_breed) and ('mix' not in cur_breed):
        df['isPurebred'][index] = 1
    else:
        df['isPurebred'][index] = 0
    if '/' in cur_breed:
       breed_lst = sorted(cur_breed.split("/"))
       df['Breed_1'][index]= breed_lst[0]
       df['Breed_2'][index]= breed_lst[1]
    else:
       df['Breed_1'][index]= cur_breed
       df['Breed_2'][index]= cur_breed
    
    ##Color
    cur_color=row['Color']
    if '/' in cur_color:
       color_lst = sorted(cur_color.split("/"))
       df['Color_1'][index]= color_lst[0]
       df['Color_2'][index]= color_lst[1]
    else:
       df['Color_1'][index]= cur_color
       df['Color_2'][index]= cur_color
    
    ##Outcome
    df['OutcomeType'][index] = convertTargetType(row['OutcomeType'])
    
        
breedDummies = findComplexDummies(df['Breed_2'],df['Breed_1'],'Breed')
colorDummies = findComplexDummies(df['Color_2'],df['Color_1'],'Color')
df = pd.get_dummies(df, columns=['AnimalType','SexuponOutcome','AgeuponOutcome','ReproductiveStatus'])
df = pd.concat([df,breedDummies,colorDummies], axis=1, sort=False)
df
df = df.drop(['Breed','Color','Breed_1','Breed_2','Color_1','Color_2'], axis=1)

In [3]:
df.to_csv("ProcessedData.csv", index=False)

In [4]:
zero = df[df.OutcomeType == 0].size
one = df[df.OutcomeType == 1].size
two = df[df.OutcomeType == 2].size
three = df[df.OutcomeType == 3].size
four = df[df.OutcomeType == 4].size
arr = []
total = zero + one + two + three + four
arr.append(zero/total)
arr.append(one/total)
arr.append(two/total)
arr.append(three/total)
arr.append(four/total)
arr

[0.40289573122825395,
 0.1790564555351865,
 0.3525010288450746,
 0.05817651240225972,
 0.007370271989225186]